In [68]:
import pandas as pd
import ast
import numpy as np
from scipy.stats import ttest_ind

In [61]:
# unit_occupancy_file = '/Users/bradley.webb/Desktop/mid_market_measurement/data/inspections/unit_occupancies.csv'
# unit_inspection_file = '/Users/bradley.webb/Desktop/mid_market_measurement/data/inspections/unit_inspections.csv'
occupancy_inspection_file = '/Users/bradley.webb/Desktop/mid_market_measurement/data/inspections/occupancy_inspections_2021_03_12.csv'
high_adoption_inspection_file = '/Users/bradley.webb/Desktop/mid_market_measurement/data/inspections/High_Adoption_Inspections_Users_-_Inspections_Engagement_2021_04_08.csv'
low_adoption_inspection_file = '/Users/bradley.webb/Desktop/mid_market_measurement/data/inspections/Low_Adoption_Inspections_Users_-_Inspections_Engagement_2021_04_08.csv'

In [31]:
# unit_df = pd.read_csv(unit_occupancy_file)
df = pd.read_csv(occupancy_inspection_file)

In [62]:
df_high = pd.read_csv(high_adoption_inspection_file)
df_low = pd.read_csv(low_adoption_inspection_file)

In [32]:
def convert(row):
    return ast.literal_eval(row)

def gen_UUID(row):
    return '{VHOST}-{PROP_ID}-{UNIT_ID}'.format(VHOST=row['VHOST'].strip(), PROP_ID=row['PROPERTY_ID'], UNIT_ID=row['UNIT_ID'])

In [33]:
df['INSPECTIONS_CONVERTED'] = df['INSPECTIONS'].apply(convert)
df['UUID'] = df.apply(gen_UUID, axis=1)

# unit_df['UUID'] = unit_df.apply(gen_UUID, axis=1)
# occupancy_inspection_df['UUID'] = occupancy_inspection_df.apply(gen_UUID, axis=1)

In [59]:
df_high.describe()

,ID,CNT_IMG,CNT_AREAS,CNT_ITEMS,CNT_NON_ZERO_LENGTH_NOTES,CNT_CHECKED_ITEMS,CNT_FLAGGED_ITEMS
count,148683.000000,148683.000000,148683.000000,148683.000000,148683.000000,148683.000000,148683.000000
mean,4441.970232,47.017097,8.552699,40.067903,5.590787,24.025820,2.492195
std,2786.347143,86.557888,5.801906,41.968787,10.186880,34.798363,6.168400
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2441.000000,1.000000,5.000000,15.000000,0.000000,0.000000,0.000000
50%,3902.000000,18.000000,8.000000,26.000000,2.000000,11.000000,0.000000
75%,5874.000000,56.000000,11.000000,43.000000,6.000000,28.000000,2.000000
max,14834.000000,2077.000000,39.000000,365.000000,148.000000,320.000000,118.000000


In [63]:
df_low.describe()

,ID,CNT_IMG,CNT_AREAS,CNT_ITEMS,CNT_NON_ZERO_LENGTH_NOTES,CNT_CHECKED_ITEMS,CNT_FLAGGED_ITEMS
count,148383.000000,148383.000000,148383.000000,148383.000000,148383.000000,148383.000000,148383.000000
mean,937.834577,31.795260,6.624910,32.143608,7.007898,12.432273,4.037450
std,1392.339867,59.483377,4.186212,35.102172,12.235643,21.971707,8.295509
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,168.000000,0.000000,4.000000,16.000000,0.000000,0.000000,0.000000
50%,456.000000,11.000000,6.000000,21.000000,2.000000,2.000000,0.000000
75%,1101.000000,37.000000,8.000000,33.000000,9.000000,16.000000,4.000000
max,9297.000000,3912.000000,60.000000,457.000000,240.000000,259.000000,185.000000


In [76]:
ttest_results = dict()
for col in df_low.columns[2:]:
    cat1 = df_low[col]
    cat2 = df_high[col]

    ttest_results[col] = ttest_ind(cat1, cat2)
# cat1 = df_low['CNT_IMG']
# cat2 = df_high['CNT_IMG']
#
# ttest_ind(cat1, cat2)
ttest_results

{'CNT_IMG': Ttest_indResult(statistic=-55.847192539770575, pvalue=0.0),
 'CNT_AREAS': Ttest_indResult(statistic=-103.83030300501646, pvalue=0.0),
 'CNT_ITEMS': Ttest_indResult(statistic=-55.81365531883691, pvalue=0.0),
 'CNT_NON_ZERO_LENGTH_NOTES': Ttest_indResult(statistic=34.30679515666044, pvalue=1.9916418421459927e-257),
 'CNT_CHECKED_ITEMS': Ttest_indResult(statistic=-108.54688878951127, pvalue=0.0),
 'CNT_FLAGGED_ITEMS': Ttest_indResult(statistic=57.61793924120315, pvalue=0.0)}

In [6]:
# pd.options.display.float_format = "{:,.2f}".format
df[df['CNT_INSPECTIONS'] >= 1].shape[0] / df.shape[0] # percent total occupancies at least 1 inspection
df[df['CNT_INSPECTIONS'] > 1].shape[0] / df.shape[0] # percent total occupancies greater than 1 inspection


0.042523162046313644

In [7]:
# occupancy_active_user
"""
Given a unit, did an inspection happen during a given occupancy?
If that occupancy has an end date, did an inspection happen during the subsequent occupancy?
"""

# df[df.groupby('UUID').agg({'CNT_INSPECTION':'sum'}) > 1]
df.groupby('UUID').count()['OCCUPANCY_ID'] > 1

UUID
a2zrealty-100-100       False
a2zrealty-101-101       False
a2zrealty-102-102        True
a2zrealty-103-103       False
a2zrealty-104-104       False
                        ...  
zukinrealty-815-2488    False
zukinrealty-815-2489    False
zukinrealty-818-2494    False
zukinrealty-819-2496    False
zukinrealty-820-2495    False
Name: OCCUPANCY_ID, Length: 1635940, dtype: bool

In [12]:
df2 = df.groupby(['VHOST', 'PROPERTY_ID']).agg({'OCCUPANCY_ID':'count', 'CNT_INSPECTIONS': 'sum'}).reset_index().rename(columns={'OCCUPANCY_ID':'CNT_OCCUPANCY', 'CNT_INSPECTIONS':'SUM_INSPECTIONS'})
df3 = df.groupby(['VHOST', 'UNIT_ID']).agg({'OCCUPANCY_ID':'count', 'CNT_INSPECTIONS': 'sum'}).reset_index().rename(columns={'OCCUPANCY_ID':'CNT_OCCUPANCY', 'CNT_INSPECTIONS':'SUM_INSPECTIONS'})
df4 = df.groupby(['VHOST']).agg({'OCCUPANCY_ID':'count', 'CNT_INSPECTIONS': 'sum'}).reset_index().rename(columns={'OCCUPANCY_ID':'CNT_OCCUPANCY', 'CNT_INSPECTIONS':'SUM_INSPECTIONS'})

In [9]:
df2[df2['SUM_INSPECTIONS'] >= 1].shape[0] / df2.shape[0] # properties with at least 1 inspection

0.26517830458485037

In [11]:
df3[df3['SUM_INSPECTIONS'] >= 1].shape[0] / df3.shape[0] # units with at least 1 inspection

0.13475066322725773

In [13]:
df4[df4['SUM_INSPECTIONS'] >= 1].shape[0] / df4.shape[0] # vhosts with at least 1 inspection

0.650089605734767

In [43]:
df3.columns
s = df3[df3['SUM_INSPECTIONS'] / df3['CNT_OCCUPANCY'] >= 1].groupby(['VHOST']).count()['UNIT_ID'] / df3.groupby(['VHOST']).count()['UNIT_ID'] <= .1

In [57]:
np.savetxt('/Users/bradley.webb/Downloads/Low_Adoption_Inspection_Users.csv', np.array(s[s].index),  delimiter=',', fmt='%s')